In [18]:
import pandas as pd
import glob
from googletrans import Translator
from google_trans_new import google_translator  
import emoji
import time
import re
from langdetect import detect
import demoji
import fasttext
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

AttributeError: partially initialized module 'nltk' has no attribute 'data' (most likely due to a circular import)

In [16]:
directory = 'targeted'
pd.set_option('display.max_colwidth', None)
#pd.set_option('display.max_rows', None)  # Show all rows
#pd.set_option('display.max_columns', None)

In [17]:
nltk.download('wordnet')
nltk.download() # only needs to be ran once
wnl = WordNetLemmatizer()
nltk.download('averaged_perceptron_tagger')

NameError: name 'nltk' is not defined

In [ ]:
def remove_emojis(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F700-\U0001F77F"  # alchemical symbols
                           u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                           u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                           u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                           u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                           u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                           u"\U0001F680-\U0001F6FF"  # CJK Unified Ideographs Extension B
                           u"\U0001F600-\U0001F64F"  # CJK Unified Ideographs
                           u"\U0001F300-\U0001F5FF"  # Miscellaneous Symbols and Pictographs"
                           u"\U0001F000-\U0001F02F"  # Mahjong Tiles
                           u"\U0001F004-\U0001F0CF"  # Domino Tiles
                           u"\U0001F110-\U0001F251"  # Playing Cards
                           u"\U0001F600-\U0001F64F"  # Emoticons (faces)
                           u"\U0001F910-\U0001F91F"  # Emojis (animal faces)
                           u"\U0001F920-\U0001F927"  # Emojis (human faces)
                           u"\U0001F930-\U0001F939"  # Emojis (clothing)
                           u"\U0001F950-\U0001F95F"  # Emojis (food)
                           u"\U0001F980-\U0001F991"  # Emojis (objects)
                           u"\U0001F9C0-\U0001F9C0"  # Emojis (symbols)
                           "]+", flags=re.UNICODE)

    return emoji_pattern.sub(r'', text)



In [ ]:
t = Translator()

translations = []
test_file = glob.glob(directory + "/*.xlsx")

test_list = []
for file in test_file:
    df = pd.read_excel(file, sheet_name='Tweets')
    test_list.append(df)


# Concatenate the DataFrames in test_list into a single DataFrame
combined_df = pd.concat(test_list, axis=0, ignore_index=True)
combined_df = combined_df.drop(['Binders', 'Tweet value', 'Likes count', 'Retweet count'], axis = 1)
combined_df['Tweet'] = combined_df['Tweet'].str.replace("’", "'")
combined_df['Tweet'] = combined_df['Tweet'].str.replace("‘", "'")
combined_df['Tweet'] = combined_df['Tweet'].str.replace("“", "\"")
combined_df['Tweet'] = combined_df['Tweet'].str.replace("”", "\"")
combined_df['Tweet'] = combined_df['Tweet'].str.replace("‼️", "!!")
combined_df['Tweet'] = combined_df['Tweet'].str.replace("…", "...")
combined_df['Tweet'] = combined_df['Tweet'].str.replace("⁃", "-")
combined_df['Tweet'] = combined_df['Tweet'].str.replace("☹️", "")
combined_df['Tweet'] = combined_df['Tweet'].apply(remove_emojis)



language_model = fasttext.load_model('lid.176.bin') 

#initalize already to English
combined_df['Is_English'] = '1'

for index, row in combined_df.iterrows():
    text = row['Tweet']
    text = text.replace('\n', ' ')
    try:
        prediction = language_model.predict(text)
        detected_language = prediction[0][0].replace("__label__", "")
        
        # Define a confidence threshold to consider a language as detected
        confidence_threshold = 0.8 
        if prediction[1][0] >= confidence_threshold:
            if detected_language == 'en':
                combined_df.at[index, 'Is_English'] = '1'  # English
            else:
                combined_df.at[index, 'Is_English'] = '0'  # Non-English
    except Exception as e:
        print(f"Error detecting language: {e}")

mask = combined_df['Is_English'].str.contains('0', case=False)


# Use the mask to filter rows that match the condition
filtered_rows = combined_df[mask]

filtered_rows.to_csv('filtered.csv')

df = pd.read_csv('filtered.csv')
# Extract the 'Tweet' column
texts = df['Tweet'].tolist()

batch_size = 10  

translations = []

for i in range(0, len(texts), batch_size):
    text_batch = texts[i:i + batch_size]
    translated_batch = []

    for text in text_batch:
        try:
            translated_text = t.translate(text, dest='en').text
            if translated_text is not None:
                translated_batch.append(translated_text)
            else:
                translated_batch.append("")
            time.sleep(1)
        except Exception as e:
            print(f"Error translating: {e}")
            translated_batch.append("Wrong") # Placeholder to determine which non-English text is causing issues
    translations.extend(translated_batch)

df['Translated'] = translations




Error translating: the JSON object must be str, bytes or bytearray, not NoneType
Error translating: the JSON object must be str, bytes or bytearray, not NoneType
Error translating: the JSON object must be str, bytes or bytearray, not NoneType
Error translating: the JSON object must be str, bytes or bytearray, not NoneType
Error translating: the JSON object must be str, bytes or bytearray, not NoneType
Error translating: the JSON object must be str, bytes or bytearray, not NoneType
Error translating: The read operation timed out
Error translating: the JSON object must be str, bytes or bytearray, not NoneType
Error translating: the JSON object must be str, bytes or bytearray, not NoneType
Error translating: the JSON object must be str, bytes or bytearray, not NoneType
Error translating: the JSON object must be str, bytes or bytearray, not NoneType
Error translating: The read operation timed out
Error translating: The read operation timed out
Error translating: The read operation timed ou

In [ ]:
df

,Unnamed: 0,User,Username,UTC Date,Tweet,Permalink,Is_English,Translated
0,738,Oceanman.Mp4,@Oceanman_mp4,2023-01-30 19:18:02,"@dom_vom_gym Fühlt sich ja mal giga targeted an nachdem ich heute genau deswegen beim arzt war, aber ich hab halt tatsächlich testomangel LMFAO",https://www.twitter.com/user/status/1620139293426749441,0,"@dom_vom_gym feels Giga Targeted after I was at the doctor today, but I actually have a test macker LMFAO"
1,1416,Birke,@482fj64,2023-01-30 19:00:31,"RT @spektrum: Krebszellen gezielt anzugreifen, erhöht die Heilungschancen enorm. Mit »targeted therapies« können Ärztinnen und Ärzte bestimmte Tumoren so genau treffen, dass der Rest des Körpers weitgehend verschont bleibt. #krebs #weltkrebstag @DKFZ @XgegenKrebs https://t.co/MRNKSeyJ9K",https://www.twitter.com/user/status/1620134882361110529,0,"RT @spectrum: to target cancer cells in a targeted manner, increases the chances of recovery enormously.With ""Targeted Therapy"", doctors can hit certain tumors so precisely that the rest of the body is largely spared.#Krebs #weltkrebstag @dkfz @xGeKrebs https://t.co/mrnkseyj9k"
2,1578,Annette Doerfel,@AnnetteDoerfel,2023-01-30 18:56:31,"RT @spektrum: Krebszellen gezielt anzugreifen, erhöht die Heilungschancen enorm. Mit »targeted therapies« können Ärztinnen und Ärzte bestimmte Tumoren so genau treffen, dass der Rest des Körpers weitgehend verschont bleibt. #krebs #weltkrebstag @DKFZ @XgegenKrebs https://t.co/MRNKSeyJ9K",https://www.twitter.com/user/status/1620133876755738625,0,"RT @spectrum: to target cancer cells in a targeted manner, increases the chances of recovery enormously.With ""Targeted Therapy"", doctors can hit certain tumors so precisely that the rest of the body is largely spared.#Krebs #weltkrebstag @dkfz @xGeKrebs https://t.co/mrnkseyj9k"
3,1966,VTV Gujarati News and Beyond,@VtvGujarati,2023-01-30 18:47:33,"લ્યો બોલો.. લગ્ન પ્રસંગમાં જવાનું હોવાથી દાગીના લેવા ગયાં, ઘરે ચોરે આવીને ચાર ચેઈન છૂમંતર કરી નાખી, મેઘાણીનગરનો કિસ્સો\n#Ahmedabad #VtvGujarati\n\nhttps://t.co/q0kMTF9vEy",https://www.twitter.com/user/status/1620131618588606464,0,"Speak Lyo .. Since going to the wedding, the jewelry went to pick up, the thieves came home and shot four chains, the case of Meghaninagar\n#Ahmedabad #vtvgujarati\n\nhttps://t.co/Q0KMTF9vey"
4,3000,Per-Ivar Kloen,@pikloen,2023-01-30 18:22:44,"@edelman_vincent @cbokhove @researchED_BE @researchED_NL In mijn beleving is het een genuanceerd proces. Grofweg 'targeted' voor het onthouden en geeft informatie over het leren, 'holistic' voor het verbinden van kennis en werkt interesse en motivatie in de hand.",https://www.twitter.com/user/status/1620125374389813249,0,"@edelman_vincent @cbokhove @researched_be @researched_NL In my experience it is a nuanced process.Roughly 'targeted' for remembering and gives information about learning, 'holistic' for connecting knowledge and encourages interest and motivation."
...,...,...,...,...,...,...,...,...
1994,458789,B V Prasad Reddy,@bvprasadreddy,2023-01-20 01:31:51,"నీచులు\nసమాజంలో\nచెడు నీచం పెంచడం కోసం,\nమంచిని తగ్గించడం కోసం&amp;\nTargeted personsను ఇబ్బంది పెట్టడం కోసం\nఒక vaccum, situations things create చేసి పెడితే\nనాయకులు అనే వారు doubt discuss debate చేసి\nఅసలెందుకిదంతా అని\nమంచి చెడు విచక్షణ\nవిషయ పరిజ్ఞానం\nఇంగితజ్ఞానంతో\nఆపాలి గానీ\nనీచుల మాయలో",https://www.twitter.com/user/status/1616247098974040064,0,"The miseries\nIn society\nFor raising the bad miserable,\nFor reducing the good & amp;\nFor embarrassing targeted persons\nIf a vaccum, SITUATIONS THINGS CREATE\nLeaders Doubt Discusss Debate\nAll right\nGood bad discretion\nSubject knowledge\nWith common sense\nStop\nIn the magic of the nihils"
1995,458852,🐨❤🍯🧸🌸🍡🌈🌟,@eiLoJDaPKooL,2023-01-20 01:23:00,"RT @inspoPageant: HIT THE POINT/ \n\nในรอบ closed-door interview #MissUniverseThailand2022 จากคำตอบของ R'Bonney Nola #MissUniverse2022 น่าจะทำให้น้อง แอนชิลี ได้รับคำตอบที่เคยถาม นิโคลีน แล้วนะว่า \n\nเพราะเหตุใดถึงต้องตอบในแบบ Beauty Queen ถึงจะมง \n\n""Get to the point, h

In [ ]:
# Some texts do not end up translating to English
df2 = df[df['Translated'].str.contains("Wrong")]
df2['Tweet'] = df2['Tweet'].str.replace("☹️", "")
df2.loc[:, 'Tweet'] = df2['Tweet'].apply(remove_emojis) #Emojis not removed for some reason
for index, row in df2.iterrows():
    text = row['Tweet']
    try:
        translated = t.translate(text, dest='en')  
        if translated:
            translated_text = translated.text
            df2.loc[index, 'Translated'] = translated_text
        else:
            df2.loc[index, 'Translated'] = "Translation Failed"
    except Exception as e:
        print(f"Error translating: {e}")
        df2.loc[index, 'Translated'] = "Translation Error"


/var/folders/4_/rf44p2zs3ld9skyp7yxhcm8r0000gn/T/ipykernel_14977/2040865592.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Tweet'] = df2['Tweet'].str.replace("☹️", "")
/Users/BrianPark/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/Users/BrianPark/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [ ]:
df2

,Unnamed: 0,User,Username,UTC Date,Tweet,Permalink,Is_English,Translated
297,85662,simaneko (Targeted Individuals),@simaneko18,2023-01-27 20:11:00,日本社会は組織犯罪のインフラを防犯のネットワークと偽り、ターゲットの民間人にあらゆる遠隔手法による精神虐待で長期加害する。\n拷問とスローキルだが、人工衛星や対人レーダーやAIや軍事兵器が使用されている。\n怖くて仕方ないけど、国も警察も集ストテク犯は放置だし、どうしたらいいの。,https://www.twitter.com/user/status/1619065456584978433,0,"Japanese society falsifies organized crime infrastructure as a crime prevention network, for a long period of time due to mental abuse through all remote methods to target civilians.\nTorture and slow key are used, but artificial satellites, interpersonal radar, AI and military weapons are used.\nI'm scared and can't help, but the country and the police are left untouched, and what should I do?"
568,153625,Sudesh Sharma,@sudesh535,2022-07-21 03:09:09,"#भारत_के_संत Sant Shri Asharamji Bapu\nSanatan Warrior हैं। उन्हें समाज से दूर करने के लिए झूठे आरोप लगाकर आजीवन कारावास की सजा दी गई जबकि सत्य सबके सामने आ चुका है, उन्हें Intentionally Targeted किया गया है। https://t.co/yNbOLSzypu",https://www.twitter.com/user/status/1549954606632865792,0,"#Bharat_K_Sant Sant Shri Asharamji Bapu\nSanatan Warrior is.In order to remove them from the society, he has been sentenced to life imprisonment by making false allegations, while the truth has come in front of everyone, they have been targeted intentally.https://t.co/ynbolszypu"
939,204687,michelina,@michelfsoto,2023-01-28 16:47:25,Les voy a echar unos whiskas alv pa que gasto en croqueta fina y targeted a sus necesidades si les vale monda,https://www.twitter.com/user/status/1619376612214185984,0,I'm going to throw some whiskas alv that spending in fine croquette and targeted their needs if they are worth monda
942,205417,Rakesh Kumar,@RakeshK85386729,2023-01-28 16:13:50,"RT @AmarUjalaNews: S Jaishankar: 'मैं चीनी राजदूत को बुलाकर नहीं पूछता...', विदेश मंत्री ने बिना नाम लिए राहुल पर साधा निशाना\n#SJaishankar #RahulGandhi\n#indiachina #G20India\n\nhttps://t.co/ZUqxR2iqzi",https://www.twitter.com/user/status/1619368162239320064,0,"RT @Amarujalanews: S Jaishankar: 'I do not ask Chinese ambassador and ask ...', Foreign Minister targeted Rahul without naming Rahul\n#SJAISHANKAR #RAHULGANDHI\n#indichina #G20India\n\nhttps://t.co/zuqxr2iqzi"
944,205918,Nitesh Shrimali Balotra,@BalotraNitesh,2023-01-28 15:47:14,"RT @Rajasthan_Tak: गहलोत ने इशारों-इशारों में सचिन पायलट पर साधा निशाना! सत्ता में वापसी का भी बताया फॉर्मूला, जानें क्या है पूरा मामला\n#RajasthanPolitics #Election2023 #AshokGehlot #SachinPilot \nhttps://t.co/LIlm6arPAB",https://www.twitter.com/user/status/1619361468084015106,0,"RT @rajasthan_tak: Gehlot targeted Sachin Pilot in gestures!Formula told to return to power, know what is the whole matter\n#Rajasthanpolitics #Election2023 #ASHOKGEHLOT #SACHINPILOT\nhttps://t.co/lilm6arpab"
945,205953,magnolia spain🇪🇸🙊🙉🙈🇪🇸,@cibertortura_Es,2023-01-28 15:45:10,"RT @viactec: Enlaces a tres grabaciones V2k realizadas en hogares del sur de Oregón, con grabadoras colocadas en los dormitorios de las víctimas. ¡Las mejores grabaciones hasta ahora de armas electrónicas/neuronales en acción! Por favor, escucha con auriculares Thank you @Targeted_Oregon https://t.co/0VnHSx9aCe",https://www.twitter.com/user/status/1619360944207040514,0,"RT @VIACTEC: Links to three V2K recordings made in households in southern Oregon, with recorders placed in the victims's bedrooms.The best recordings so far of electronic/neuronal weapons in action!Please listen with Thank You @TarGeled_oregon https://t.co/0vnhsx9ace"
981,216199,J-HOPE REPORT BR⁷ | CLOSED,@Jhopesuporte,2022-04-30 04:21:31,LISTA DE MASS REPORT \n\nAs contas abaixo estão atacando o por palavras ofensivas após ele ter feito a publicação \n\n▪ desrespeitoso ou ofensivo \n▪ assédio direcionado\n▪ disrespectful or offensive \n▪ targeted harassment \n\n⬇️,https://www.twitter.com/user/status/1520257013988708352,0,Mass Report List\n\nThe accounts below are attacking the offensive words after he has made the publication\n\n▪ D

In [ ]:
# delete placeholders and add translated texts to originial dataframe

df = df[~df['Translated'].str.contains('Wrong')]
df = pd.concat([df, df2], axis=0, ignore_index=True)
df

,Unnamed: 0,User,Username,UTC Date,Tweet,Permalink,Is_English,Translated
0,738,Oceanman.Mp4,@Oceanman_mp4,2023-01-30 19:18:02,"@dom_vom_gym Fühlt sich ja mal giga targeted an nachdem ich heute genau deswegen beim arzt war, aber ich hab halt tatsächlich testomangel LMFAO",https://www.twitter.com/user/status/1620139293426749441,0,"@dom_vom_gym feels Giga Targeted after I was at the doctor today, but I actually have a test macker LMFAO"
1,1416,Birke,@482fj64,2023-01-30 19:00:31,"RT @spektrum: Krebszellen gezielt anzugreifen, erhöht die Heilungschancen enorm. Mit »targeted therapies« können Ärztinnen und Ärzte bestimmte Tumoren so genau treffen, dass der Rest des Körpers weitgehend verschont bleibt. #krebs #weltkrebstag @DKFZ @XgegenKrebs https://t.co/MRNKSeyJ9K",https://www.twitter.com/user/status/1620134882361110529,0,"RT @spectrum: to target cancer cells in a targeted manner, increases the chances of recovery enormously.With ""Targeted Therapy"", doctors can hit certain tumors so precisely that the rest of the body is largely spared.#Krebs #weltkrebstag @dkfz @xGeKrebs https://t.co/mrnkseyj9k"
2,1578,Annette Doerfel,@AnnetteDoerfel,2023-01-30 18:56:31,"RT @spektrum: Krebszellen gezielt anzugreifen, erhöht die Heilungschancen enorm. Mit »targeted therapies« können Ärztinnen und Ärzte bestimmte Tumoren so genau treffen, dass der Rest des Körpers weitgehend verschont bleibt. #krebs #weltkrebstag @DKFZ @XgegenKrebs https://t.co/MRNKSeyJ9K",https://www.twitter.com/user/status/1620133876755738625,0,"RT @spectrum: to target cancer cells in a targeted manner, increases the chances of recovery enormously.With ""Targeted Therapy"", doctors can hit certain tumors so precisely that the rest of the body is largely spared.#Krebs #weltkrebstag @dkfz @xGeKrebs https://t.co/mrnkseyj9k"
3,1966,VTV Gujarati News and Beyond,@VtvGujarati,2023-01-30 18:47:33,"લ્યો બોલો.. લગ્ન પ્રસંગમાં જવાનું હોવાથી દાગીના લેવા ગયાં, ઘરે ચોરે આવીને ચાર ચેઈન છૂમંતર કરી નાખી, મેઘાણીનગરનો કિસ્સો\n#Ahmedabad #VtvGujarati\n\nhttps://t.co/q0kMTF9vEy",https://www.twitter.com/user/status/1620131618588606464,0,"Speak Lyo .. Since going to the wedding, the jewelry went to pick up, the thieves came home and shot four chains, the case of Meghaninagar\n#Ahmedabad #vtvgujarati\n\nhttps://t.co/Q0KMTF9vey"
4,3000,Per-Ivar Kloen,@pikloen,2023-01-30 18:22:44,"@edelman_vincent @cbokhove @researchED_BE @researchED_NL In mijn beleving is het een genuanceerd proces. Grofweg 'targeted' voor het onthouden en geeft informatie over het leren, 'holistic' voor het verbinden van kennis en werkt interesse en motivatie in de hand.",https://www.twitter.com/user/status/1620125374389813249,0,"@edelman_vincent @cbokhove @researched_be @researched_NL In my experience it is a nuanced process.Roughly 'targeted' for remembering and gives information about learning, 'holistic' for connecting knowledge and encourages interest and motivation."
...,...,...,...,...,...,...,...,...
1994,274128,TV9 Bihar Jharkhand,@TV9Bihar,2023-01-28 07:04:06,"शराबबंदी पर मांझी ने फिर साधा निशाना, कहा- बिना 'जाम' के बिहार में नहीं आएंगे पर्यटक \n\n#SharabBandi #LiquorBan \n\n@jitanrmanjhi @NitishKumar @Jduonline @BJP4Bihar \n\nhttps://t.co/tBzGDnAzVP",https://www.twitter.com/user/status/1619229815722381313,0,"Manjhi again targeted on liquor ban, said- tourists will not come to Bihar without 'jam'\n\n#SHARABBANDI #Liquorban\n\n@jitanrmanjhi @nitishkumar @jduonline @BJP4bihar\n\nhttps://t.co/tbzgdnazvp"
1995,347456,ประกันออนไลน์ สไตล์นักบัญชี ประกันชีวิต สุขภาพ-นุช,@AppleMTL42,2023-02-04 16:10:30,@852style (2) EliteHealthPlus\n✅️ค่ารักษาเหมาจ่าย20-100ลบ/ปี /นอนห้องเดี่ยวเริ่มต้นได้ทุก รพ.\n✅️คุ้มครองโรคระบาด โรคร้ายแรง โรคทั่วไป และอุบัติเหตุ\n✅️รักษามะเร็งมุ่งเป้า(Targeted Therapy)/MRI/ฟอกไต\n⛑️มีประกันอุบัติเหตุ\nรายเดือน\nเลือกคุ้มครองทั่วโลก\n\nLinehttps://t.co/CrSPDTU7ob https://t.co/01ZaeY6wtS,https://www.twitter.com/user/status/1621904035438817284,0,"@852style (2) Elitehealthplus\n✅️ Treatment fee payment 20-100 delete/ye

In [ ]:
lemmatizer = WordNetLemmatizer()

def lemmatize_with_pos_tag(sentence):
    def pos_tagger(nltk_tag):
        if nltk_tag.startswith('J'):
            return wordnet.ADJ
        elif nltk_tag.startswith('V'):
            return wordnet.VERB
        elif nltk_tag.startswith('N'):
            return wordnet.NOUN
        elif nltk_tag.startswith('R'):
            return wordnet.ADV
        else:
            return None

    pos_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))

    wordnet_tagged = [(word, pos_tagger(tag)) for word, tag in pos_tagged]

    lemmatized_tokens = []
    for word, tag in wordnet_tagged:
        if tag is None:
            # if there is no available tag, append the token as is
            lemmatized_tokens.append(word)
        else:
            # else use the tag to lemmatize the token
            lemmatized_tokens.append(lemmatizer.lemmatize(word, tag))
    
    lemmatized_sentence = " ".join(lemmatized_tokens)
    return lemmatized_sentence

df['complex_lemmatized_column'] = df['Translated'].apply(lambda x: lemmatize_with_pos_tag(x))

In [ ]:
df

,Unnamed: 0,User,Username,UTC Date,Tweet,Permalink,Is_English,Translated,complex_lemmatized_column
0,738,Oceanman.Mp4,@Oceanman_mp4,2023-01-30 19:18:02,"@dom_vom_gym Fühlt sich ja mal giga targeted an nachdem ich heute genau deswegen beim arzt war, aber ich hab halt tatsächlich testomangel LMFAO",https://www.twitter.com/user/status/1620139293426749441,0,"@dom_vom_gym feels Giga Targeted after I was at the doctor today, but I actually have a test macker LMFAO","@ dom_vom_gym feel Giga Targeted after I be at the doctor today , but I actually have a test macker LMFAO"
1,1416,Birke,@482fj64,2023-01-30 19:00:31,"RT @spektrum: Krebszellen gezielt anzugreifen, erhöht die Heilungschancen enorm. Mit »targeted therapies« können Ärztinnen und Ärzte bestimmte Tumoren so genau treffen, dass der Rest des Körpers weitgehend verschont bleibt. #krebs #weltkrebstag @DKFZ @XgegenKrebs https://t.co/MRNKSeyJ9K",https://www.twitter.com/user/status/1620134882361110529,0,"RT @spectrum: to target cancer cells in a targeted manner, increases the chances of recovery enormously.With ""Targeted Therapy"", doctors can hit certain tumors so precisely that the rest of the body is largely spared.#Krebs #weltkrebstag @dkfz @xGeKrebs https://t.co/mrnkseyj9k","RT @ spectrum : to target cancer cell in a targeted manner , increase the chance of recovery enormously.With `` Targeted Therapy '' , doctor can hit certain tumor so precisely that the rest of the body be largely spared. # Krebs # weltkrebstag @ dkfz @ xGeKrebs http : //t.co/mrnkseyj9k"
2,1578,Annette Doerfel,@AnnetteDoerfel,2023-01-30 18:56:31,"RT @spektrum: Krebszellen gezielt anzugreifen, erhöht die Heilungschancen enorm. Mit »targeted therapies« können Ärztinnen und Ärzte bestimmte Tumoren so genau treffen, dass der Rest des Körpers weitgehend verschont bleibt. #krebs #weltkrebstag @DKFZ @XgegenKrebs https://t.co/MRNKSeyJ9K",https://www.twitter.com/user/status/1620133876755738625,0,"RT @spectrum: to target cancer cells in a targeted manner, increases the chances of recovery enormously.With ""Targeted Therapy"", doctors can hit certain tumors so precisely that the rest of the body is largely spared.#Krebs #weltkrebstag @dkfz @xGeKrebs https://t.co/mrnkseyj9k","RT @ spectrum : to target cancer cell in a targeted manner , increase the chance of recovery enormously.With `` Targeted Therapy '' , doctor can hit certain tumor so precisely that the rest of the body be largely spared. # Krebs # weltkrebstag @ dkfz @ xGeKrebs http : //t.co/mrnkseyj9k"
3,1966,VTV Gujarati News and Beyond,@VtvGujarati,2023-01-30 18:47:33,"લ્યો બોલો.. લગ્ન પ્રસંગમાં જવાનું હોવાથી દાગીના લેવા ગયાં, ઘરે ચોરે આવીને ચાર ચેઈન છૂમંતર કરી નાખી, મેઘાણીનગરનો કિસ્સો\n#Ahmedabad #VtvGujarati\n\nhttps://t.co/q0kMTF9vEy",https://www.twitter.com/user/status/1620131618588606464,0,"Speak Lyo .. Since going to the wedding, the jewelry went to pick up, the thieves came home and shot four chains, the case of Meghaninagar\n#Ahmedabad #vtvgujarati\n\nhttps://t.co/Q0KMTF9vey","Speak Lyo .. Since go to the wedding , the jewelry go to pick up , the thief come home and shot four chain , the case of Meghaninagar # Ahmedabad # vtvgujarati http : //t.co/Q0KMTF9vey"
4,3000,Per-Ivar Kloen,@pikloen,2023-01-30 18:22:44,"@edelman_vincent @cbokhove @researchED_BE @researchED_NL In mijn beleving is het een genuanceerd proces. Grofweg 'targeted' voor het onthouden en geeft informatie over het leren, 'holistic' voor het verbinden van kennis en werkt interesse en motivatie in de hand.",https://www.twitter.com/user/status/1620125374389813249,0,"@edelman_vincent @cbokhove @researched_be @researched_NL In my experience it is a nuanced process.Roughly 'targeted' for remembering and gives information about learning, 'holistic' for connecting knowledge and encourages interest and motivation.","@ edelman_vincent @ cbokhove @ researched_be @ researched_NL In my experience it be a nuanced process.Roughly 'targeted ' for remember and give information about learning , 'holistic '

In [ ]:
eng_df = combined_df[combined_df['Is_English'].str.contains('1', case=False)]
eng_df['complex_lemmatized_column'] = eng_df['Tweet'].apply(lambda x: lemmatize_with_pos_tag(x))

NameError: name 'combined_df' is not defined